# Neural DSL on Google Colab

This notebook demonstrates how to use Neural DSL in Google Colab environments.

## 1. Installation

First, we need to install Neural DSL:

In [ ]:
# Install Neural DSL
!pip install git+https://github.com/Lemniscate-SHA-256/Neural.git

# Install pyngrok for tunneling
!pip install pyngrok

## 2. Check GPU Availability

Let's check if we have a GPU available:

In [ ]:
# Check for GPU
!nvidia-smi

## 3. Import Neural Cloud Module

Now we can import the Neural cloud module:

In [ ]:
from neural.cloud.cloud_execution import CloudExecutor

# Initialize the cloud executor
executor = CloudExecutor()
print(f"Detected environment: {executor.environment}")
print(f"GPU available: {executor.is_gpu_available}")

## 4. Define a Neural DSL Model

Let's define a simple CNN model for CIFAR-10:

In [ ]:
dsl_code = """
network Cifar10CNN {
    input: (32, 32, 3)
    layers:
        Conv2D(32, (3, 3), "relu", padding="same")
        Conv2D(32, (3, 3), "relu")
        MaxPooling2D((2, 2))
        Dropout(0.25)
        
        Conv2D(64, (3, 3), "relu", padding="same")
        Conv2D(64, (3, 3), "relu")
        MaxPooling2D((2, 2))
        Dropout(0.25)
        
        Flatten()
        Dense(512, "relu")
        Dropout(0.5)
        Dense(10, "softmax")
    loss: "categorical_crossentropy"
    optimizer: Adam(learning_rate=0.001)
}
"""

print(dsl_code)

## 5. Compile the Model

Now we can compile the model to PyTorch code:

In [ ]:
# Compile to PyTorch
pt_model_path = executor.compile_model(dsl_code, backend='pytorch')
print(f"Model compiled to: {pt_model_path}")

# Display the generated code
with open(pt_model_path, 'r') as f:
    print(f.read())

## 6. Run the Model

Let's run the compiled model:

In [ ]:
# Run the model
results = executor.run_model(pt_model_path, dataset='CIFAR10', epochs=5, batch_size=128)

if results['success']:
    print("Model execution successful!")
else:
    print(f"Model execution failed: {results['error']}")

## 7. Visualize the Model

We can also visualize the model architecture:

In [ ]:
# Visualize the model
viz_path = executor.visualize_model(dsl_code, output_format='png')

# Display the visualization
from IPython.display import Image
Image(filename=viz_path)

## 8. Start the NeuralDbg Dashboard

We can also start the NeuralDbg dashboard with an ngrok tunnel:

In [ ]:
# Start the dashboard
dashboard_info = executor.start_debug_dashboard(dsl_code, backend='pytorch', setup_tunnel=True)

print(f"Dashboard URL: {dashboard_info['dashboard_url']}")
print(f"Tunnel URL: {dashboard_info['tunnel_url']}")

# Display a clickable link
from IPython.display import HTML
HTML(f"<a href='{dashboard_info['tunnel_url']}' target='_blank'>Open NeuralDbg Dashboard</a>")

## 9. Start the No-Code Interface

Finally, we can start the Neural No-Code interface:

In [ ]:
# Start the No-Code interface
nocode_info = executor.start_nocode_interface(setup_tunnel=True)

print(f"No-Code Interface URL: {nocode_info['interface_url']}")
print(f"Tunnel URL: {nocode_info['tunnel_url']}")

# Display a clickable link
from IPython.display import HTML
HTML(f"<a href='{nocode_info['tunnel_url']}' target='_blank'>Open Neural No-Code Interface</a>")

## 10. Advanced: HPO in Colab

Let's try hyperparameter optimization in Colab:

In [ ]:
# Define a model with HPO
hpo_dsl_code = """
network Cifar10HPO {
    input: (32, 32, 3)
    layers:
        Conv2D(HPO(choice(32, 64, 128)), (3, 3), "relu", padding="same")
        MaxPooling2D((2, 2))
        Dropout(HPO(range(0.1, 0.5, step=0.1)))
        
        Conv2D(64, (3, 3), "relu")
        MaxPooling2D((2, 2))
        
        Flatten()
        Dense(HPO(choice(128, 256, 512)), "relu")
        Dropout(0.5)
        Dense(10, "softmax")
    loss: "categorical_crossentropy"
    optimizer: Adam(learning_rate=HPO(log_range(0.0001, 0.01)))
    train {
        epochs: 5
        batch_size: HPO(choice(32, 64, 128))
        search_method: "bayesian"
    }
}
"""

# Compile with HPO
hpo_model_path = executor.compile_model(hpo_dsl_code, backend='tensorflow')
print(f"HPO model compiled to: {hpo_model_path}")

# Run HPO
hpo_results = executor.run_model(hpo_model_path, dataset='CIFAR10')

if hpo_results['success']:
    print("HPO execution successful!")
else:
    print(f"HPO execution failed: {hpo_results['error']}")

## 11. Cleanup

When you're done, you can clean up the temporary files and processes:

In [ ]:
# Clean up
executor.cleanup()